In [6]:
! cd ../data/; ls; pwd

CompTot-Env.rda		    maildir	      prjCCppJavaCshVb.db
enron_mail_20150507.tar.gz  ny_salaries.json
/home/jovyan/work/notebooks/data


In [1]:
import os
import shutil

os.getcwd()

'/home/jovyan/work/notebooks/private'

In [2]:
os.chdir('/home/jovyan/work/notebooks/data/enron_emails')

In [118]:
names_employees = os.listdir('./maildir/')
len(names_employees)

150

In [138]:
employee = sc.parallelize(names_employees)

In [13]:
import os
import shutil

src = r'/home/jovyan/work/notebooks/data/enron_emails/maildir/'
dest = r'/home/jovyan/work/notebooks/data/enron_emails/combined/'

src_dirs = os.listdir(src)
for dir in src_dirs:
    dir_full = os.path.join(src,dir,'all_documents')
    if (os.path.isdir(dir_full)):
        src_files = os.listdir(dir_full)
        for file_name in src_files:
            file_name_full = os.path.join(dir_full, file_name)
            name_new = dir+'_'+file_name
            file_name_new = os.path.join(dir_full, name_new)
            if (os.path.isfile(file_name_full)):
                os.rename(file_name_full, file_name_new)
                shutil.copy(file_name_new, dest)

In [4]:
from pyspark import SparkContext
sc = SparkContext("local")

In [5]:
! ls combined/allen-p_[1-9].

combined/allen-p_1.  combined/allen-p_4.  combined/allen-p_7.
combined/allen-p_2.  combined/allen-p_5.  combined/allen-p_8.
combined/allen-p_3.  combined/allen-p_6.  combined/allen-p_9.


In [15]:
# only get the emails from authors with name starting with 'a'
rddEmails = sc.textFile('file:///home/jovyan/work/notebooks/data/enron_emails/combined/allen-p_[1-9][1-9].')

In [16]:
rddEmails.getNumPartitions()

628

In [17]:
rddEmails.count()

27717

In [21]:
rddEmails.take(10)

['Message-ID: <9144576.1075855665395.JavaMail.evans@thyme>',
 'Date: Wed, 13 Dec 2000 11:02:00 -0800 (PST)',
 'From: arsystem@mailman.enron.com',
 'To: phillip.k.allen@enron.com',
 'Subject: Your Approval is Overdue: Access Request for',
 ' barry.tycholiz@enron.com',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: ARSystem@mailman.enron.com']

In [22]:
import re

In [68]:
flattend = rddEmails.filter(lambda line: len(line)>0 ).map(lambda line: line.lower())

In [79]:
From = flattend.filter(lambda x: 'from:' in x)
To = flattend.filter(lambda x: 'to:' in x)

In [80]:
From.count()

1357

In [83]:
From.take(10)

['from: arsystem@mailman.enron.com',
 'x-from: arsystem@mailman.enron.com',
 'from: phillip.allen@enron.com',
 'x-from: phillip k allen',
 'from: phillip.allen@enron.com',
 'x-from: phillip k allen',
 'from: phillip.allen@enron.com',
 'x-from: phillip k allen',
 'from: messenger@ecm.bloomberg.com',
 'x-from: "bloomberg.com" <messenger@ecm.bloomberg.com>']

In [121]:
FromAddr = From.flatMap(lambda x: x.split(': ')).\
            filter(lambda x: 'from' not in x).\
            filter(lambda x: 'x-from' not in x).\
            filter(lambda x: '@' not in x).\
            distinct()

In [129]:
FromAddr_1 = FromAddr.flatMap(lambda line: re.split('\d+', line)).\
                    filter(lambda x: len(x)>7)

In [131]:
FromAddr_1.take(5)

[' william kelly                           ',
 ' john d suarez                           ',
 ' colleen sullivan                           ',
 ' rebecca w cantrell                           ',
 ' rebecca w cantrell                           ']

In [133]:
FromAddr_1.count()

78

In [139]:
diff = FromAddr_1.subtract(employee)
diff.count()

78

In [142]:
intr = FromAddr_1.intersection(employee)
intr.count()

0

In [81]:
To.count()

1541

In [116]:
ToAddr = To.flatMap(lambda x: x.split(': ')).\
            flatMap(lambda x: x.split(',')).\
            filter(lambda x: 'to' not in x).\
            filter(lambda x: 'x-to' not in x).\
            filter(lambda x: '@' not in x).\
            distinct()

In [117]:
ToAddr.take(10)

['',
 ' doug gilbert-smith',
 ' joseph deffner',
 ' phillip k=',
 ' christi l nicolay',
 'david w delainey',
 'jeffrey c gossett',
 ' barry tycholiz',
 ' ',
 'andrea richards']

In [132]:
ToAddr.count()

306

In [141]:
sub = ToAddr.subtract(employee)
sub.count()

306

In [143]:
intr = ToAddr.intersection(employee)
intr.count()

0